In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2 
from skimage import exposure
from tqdm import tqdm

In [2]:
def normalize(X):
    for i in tqdm(range(X.shape[-1])):
        X[..., i] = exposure.rescale_intensity(X[..., i], out_range=(-1, 1))
        X[..., i] = exposure.equalize_adapthist(X[..., i])
    return X

In [3]:
files = glob.glob("keren/*.npz")
files = np.sort(np.array(files))

In [4]:
def extract_cell_label(image, mask, ctypes):
    cells_ind = np.unique(mask[:, :, 1])
    cell_mask = mask[:, :, 1]
    cell_size=64
    cells = []
    labels = []
    for cell_ind in tqdm(cells_ind):
        label = ctypes[cell_ind]
        labels.append(label)
        x, y = np.where(cell_mask==cell_ind)
        
        if len(x)==0 or len(y)==0:
            continue
        cell = image[x, y]
        x1, y1, x2, y2 = x.min(), y.min(), x.max(), y.max()
        single_cell_mask = np.zeros((x2-x1, y2-y1))
        single_cell_mask = (cell_mask[x1:x2, y1:y2]==cell_ind)
        
        cell_image = image[x1:x2, y1:y2]
        cell_image = cell_image * np.repeat(np.expand_dims(single_cell_mask, axis=2), 51, axis=2)
        
        cell_image = cv2.resize(cell_image, (cell_size, cell_size))
        cells.append(cell_image)
        
    return cells, labels

In [5]:

data = np.load(files[0], allow_pickle=True)
image = data["X"][0]
mask = data["y"][0]
eq_X = normalize(image)
ctypes = data["cell_types"].item()
cells, labels = extract_cell_label(image, mask, ctypes)

cells = np.array(cells)
labels = np.array(labels)

100%|███████████████████████████████████████| 5283/5283 [01:19<00:00, 66.12it/s]


In [7]:
len(labels)

5283

In [8]:
np.save("cell_channels.npy", cells, allow_pickle=True)
np.save("cell_labels.npy", labels, allow_pickle=True)